<em style="text-align:center">Copyright Iván Pinar Domínguez</em>

## 0.Importar librerías iniciales e instancia de modelo de chat

In [15]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools,initialize_agent,AgentType,create_react_agent,AgentExecutor
f = open('../OpenAI_key.txt')
api_key = f.read()
llm = ChatOpenAI(openai_api_key=api_key,temperature=0) #Recomendable temperatura a 0 para que el LLM no sea muy creativo, vamos a tener muchas herramientas a nuestra disposición y queremos que sea más determinista

## 1.Cargamos la BD Vectores y el compresor

In [16]:
#Podríamos establecer que tuviera memoria
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history") #ponemos una denominada clave a la memoria "chat_history"

In [17]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

funcion_embedding = OpenAIEmbeddings(openai_api_key=api_key)
persist_path="../Bloque 3_Conectores de Datos/ejemplosk_embedding_db"
vector_store_connection = SKLearnVectorStore(embedding=funcion_embedding, persist_path=persist_path, serializer="parquet")
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vector_store_connection.as_retriever())

## 2.Creamos una nueva herramienta a partir de la BD Vectorial para obtener resultados optimizados

In [18]:
from langchain.agents import tool

In [19]:
@tool
def consulta_interna(text: str) -> str:
    '''Retorna respuestas sobre la historia de España. Se espera que la entrada sea una cadena de texto
    y retorna una cadena con el resultado más relevante. Si la respuesta con esta herramienta es relevante,
    no debes usar ninguna herramienta más ni tu propio conocimiento como LLM'''
    compressed_docs = compression_retriever.invoke(text)
    resultado = compressed_docs[0].page_content
    return resultado

In [20]:
tools = load_tools(["wikipedia","llm-math"],llm=llm)

In [21]:
tools=tools+[consulta_interna]

## 3.Creamos el agente y lo ejecutamos 

In [22]:
agent = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,memory=memory,verbose=True)

In [23]:
agent.invoke("¿Qué periodo abarca cronológicamente en España el siglo de oro?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: consulta_interna
Action Input: ¿Qué periodo abarca cronológicamente en España el siglo de oro?
Observation: El Siglo de Oro español fue un período floreciente en las artes y las letras hispanas que abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.
Thought:Do I need to use a tool? No
AI: El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.

> Finished chain.


{'input': '¿Qué periodo abarca cronológicamente en España el siglo de oro?',
 'chat_history': '',
 'output': 'El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.'}

In [24]:
agent.invoke("¿Qué pasó durante la misma etapa en Francia?") #Gracias a tener memoria compara en esas fechas qué ocurrió en Francia



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: consulta_interna
Action Input: ¿Qué pasó durante el Siglo de Oro en Francia?
Observation: Siglo de Oro
Artículo principal: Siglo de Oro

Miguel de Cervantes Saavedra
El Siglo de Oro español fue un período floreciente en las artes y las letras hispanas que abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.
Thought:Do I need to use a tool? No
AI: Durante la misma etapa en Francia, se conoce como el Siglo de Oro de la literatura francesa, que abarca aproximadamente desde el siglo XVI hasta el siglo XVII. Durante este período, hubo un florecimiento de la literatura, la poesía, el teatro y las artes en general en Francia. Grandes escritores como Pierre de Ronsard, Joachim du Bellay, Michel de Montaigne y Pierre Corneille destacaron en este período.

> Finished chain.


{'input': '¿Qué pasó durante la misma etapa en Francia?',
 'chat_history': 'Human: ¿Qué periodo abarca cronológicamente en España el siglo de oro?\nAI: El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.',
 'output': 'Durante la misma etapa en Francia, se conoce como el Siglo de Oro de la literatura francesa, que abarca aproximadamente desde el siglo XVI hasta el siglo XVII. Durante este período, hubo un florecimiento de la literatura, la poesía, el teatro y las artes en general en Francia. Grandes escritores como Pierre de Ronsard, Joachim du Bellay, Michel de Montaigne y Pierre Corneille destacaron en este período.'}

In [25]:
agent.invoke("¿Cuáles son las marcas de vehículos más famosas hoy en día?") #Pregunta que no podemos responder con nuestra BD Vectorial



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: List of automobile manufacturers
Observation: Page: List of automobile manufacturers
Summary: Automobile manufacturers are companies and organizations that produce motor vehicles. Many of these companies are still in business, and many of the companies are defunct.  Only companies that have articles on Wikipedia are included in this list. The list is sorted by country of origin.

Page: List of automobile manufacturers of China
Summary: There are hundreds of automobile manufacturers operating in China, the country with the largest automotive industry in the world. It includes state-owned manufacturers, privately owned manufacturers, foreign manufacturers, and joint ventures between local and foreign manufacturers. See article Automobile manufacturers and brands of China for the more detailed introduction.



Page: List of automobile manufacturers of Germany
Summary: 
Thought:Do 

{'input': '¿Cuáles son las marcas de vehículos más famosas hoy en día?',
 'chat_history': 'Human: ¿Qué periodo abarca cronológicamente en España el siglo de oro?\nAI: El Siglo de Oro español abarca cronológicamente desde la publicación de la Gramática castellana de Nebrija en 1492 hasta la muerte de Calderón en 1681.\nHuman: ¿Qué pasó durante la misma etapa en Francia?\nAI: Durante la misma etapa en Francia, se conoce como el Siglo de Oro de la literatura francesa, que abarca aproximadamente desde el siglo XVI hasta el siglo XVII. Durante este período, hubo un florecimiento de la literatura, la poesía, el teatro y las artes en general en Francia. Grandes escritores como Pierre de Ronsard, Joachim du Bellay, Michel de Montaigne y Pierre Corneille destacaron en este período.',
 'output': 'Some of the most famous vehicle brands today include Toyota, Volkswagen, Ford, Honda, Chevrolet, Nissan, BMW, Mercedes-Benz, and Audi, among others. These brands are well-known for their quality, innova